In [14]:
import os
from torchrl.envs import GymWrapper
from torchrl.envs import GymEnv, TransformedEnv, VecNorm
import copy
import tempfile

import gymnasium as gym
import torch

from matplotlib import pyplot as plt
from tensordict import TensorDictBase

from tensordict.nn import TensorDictModule, TensorDictSequential
from torch import multiprocessing

from torchrl.collectors import SyncDataCollector
from torchrl.data import LazyMemmapStorage, RandomSampler, ReplayBuffer

from torchrl.envs import (
    check_env_specs,
    ExplorationType,
    PettingZooEnv,
    RewardSum,
    set_exploration_type,
    TransformedEnv,
    VmasEnv,
)

from torchrl.modules import (
    AdditiveGaussianWrapper,
    MultiAgentMLP,
    ProbabilisticActor,
    TanhDelta,
)

from torchrl.objectives import DDPGLoss, SoftUpdate, ValueEstimators

from torchrl.record import CSVLogger, PixelRenderTransform, VideoRecorder

from tqdm import tqdm

In [15]:
device = "mps"

In [30]:
base_env = GymEnv("intersection-multi-agent-v0", device=device)

In [26]:
print("action_spec:", base_env.full_action_spec)
print("reward_spec:", base_env.full_reward_spec)

action_spec: CompositeSpec(
    action: OneHotDiscreteTensorSpec(
        shape=torch.Size([2, 3]),
        space=DiscreteBox(n=3),
        device=mps,
        dtype=torch.int64,
        domain=discrete), device=mps, shape=torch.Size([]))
reward_spec: CompositeSpec(
    reward: UnboundedContinuousTensorSpec(
        shape=torch.Size([1]),
        space=None,
        device=mps,
        dtype=torch.float32,
        domain=continuous), device=mps, shape=torch.Size([]))


In [18]:
print("done_spec:", base_env.full_done_spec)
print("observation_spec:", base_env.observation_spec)

done_spec: CompositeSpec(
    done: DiscreteTensorSpec(
        shape=torch.Size([1]),
        space=DiscreteBox(n=2),
        device=mps,
        dtype=torch.bool,
        domain=discrete),
    terminated: DiscreteTensorSpec(
        shape=torch.Size([1]),
        space=DiscreteBox(n=2),
        device=mps,
        dtype=torch.bool,
        domain=discrete),
    truncated: DiscreteTensorSpec(
        shape=torch.Size([1]),
        space=DiscreteBox(n=2),
        device=mps,
        dtype=torch.bool,
        domain=discrete), device=mps, shape=torch.Size([]))
observation_spec: CompositeSpec(
    observation: UnboundedContinuousTensorSpec(
        shape=torch.Size([2, 5, 5]),
        space=None,
        device=mps,
        dtype=torch.float32,
        domain=continuous), device=mps, shape=torch.Size([]))


In [19]:
print("action_keys:", base_env.action_keys)
print("reward_keys:", base_env.reward_keys)
print("done_keys:", base_env.done_keys)

action_keys: ['action']
reward_keys: ['reward']
done_keys: ['done', 'terminated', 'truncated']


In [20]:
# env = TransformedEnv(
#     base_env,
#     RewardSum(
#         in_keys=base_env.reward_keys,
#         reset_keys=["_reset"] * len(base_env.group_map.keys()),
#     ),
# )
env = base_env

In [22]:
check_env_specs(env)

AssertionError: 

In [ ]:
frames_per_batch = 1_000  # Number of team frames collected per sampling iteration
n_iters = 10  # Number of sampling and training iterations
total_frames = frames_per_batch * n_iters

# We will stop training the evaders after this many iterations,
# should be 0 <= iteration_when_stop_training_evaders <= n_iters
iteration_when_stop_training_evaders = n_iters // 2

# Replay buffer
memory_size = 1_000_000  # The replay buffer of each group can store this many frames

# Training
n_optimiser_steps = 100  # Number of optimisation steps per training iteration
train_batch_size = 128  # Number of frames trained in each optimiser step
lr = 3e-4  # Learning rate
max_grad_norm = 1.0  # Maximum norm for the gradients

# DDPG
gamma = 0.99  # Discount factor
polyak_tau = 0.005  # Tau for the soft-update of the target network

max_steps = 100  # Environment steps before done

n_chasers = 2
n_evaders = 1
n_obstacles = 2

In [ ]:
n_rollout_steps = 5
rollout = env.rollout(n_rollout_steps)
print(f"rollout of {n_rollout_steps} steps:", rollout)
print("Shape of the rollout TensorDict:", rollout.batch_size)
env.close()